# Importación de librerias

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

# Importación de datos

join: tabla de prestamos junto a la información de los ítems después de la limpieza y extracción de atributos adicionales. 

In [18]:
#importamos la tabla de join 
join = pd.read_json('https://www.dropbox.com/s/i1komhf7u1c4y95/joinTablas.json?dl=1')


In [170]:
#Eliminamos algunas columnas que no nos interesan para este notebook
join = join.drop(["Fecha","Dewey","Facultad","Temas","Union","TipoItem"], axis=1)
#Ejemplos y columnas del dataframe
display(join.head(3))
display(join.columns)

,RowID,IDItem,NumeroUbicacion,Ubicacion,Llave,Programa,IDUsuario,Year,Signatura,FechaCreacion,Autor,Titulo,AnioPublicacion,DeweyEspecifico,TemaDewey,DeweyUnidad,DeweyDecena,DeweyCentena
0,Row0,80000005327627,720.98 A71S,COL-GRAL-2,866245,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.98,2013,"Arango Cardinal, Silvia 1948-",Ciudad y arquitectura seis generaciones que co...,2012.0,720.98,Arquitectura latinoamericana,720,720,700
1,Row1,80000001191496,720.9861 A71,COL-GRAL-2,309945,CARRERA DE ARQUITECTURA,c361c772cd0220bb16dcfb2ad803e4903334ab94,2019,720.9861,1993,"Arango Cardinal, Silvia 1948-",Historia de la arquitectura en Colombia Silvia...,1993.0,720.9861,Arquitectura colombiana,720,720,700
2,Row2,80000004979759,540 CH15Q 2010,COL-GRAL-3,822727,CARRERA DE ARQUITECTURA,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,2019,540,2011,"Chang, Raymond",Química Raymond Chang ; revisión técnica Rosa ...,2010.0,540,Química,540,540,500


Index(['RowID', 'IDItem', 'NumeroUbicacion', 'Ubicacion', 'Llave', 'Programa',
       'IDUsuario', 'Year', 'Signatura', 'FechaCreacion', 'Autor', 'Titulo',
       'AnioPublicacion', 'DeweyEspecifico', 'TemaDewey', 'DeweyUnidad',
       'DeweyDecena', 'DeweyCentena'],
      dtype='object')

# Preparación de los datos
Crearemos la columna de pesos así como el dataframe que identificará los gustos y preferencias de los usuarios. Estos dataframes se crearán teniendo en cuenta 3 columnas diferentes:
* DeweyUnidad 
* DeweyDecena
* DeweyCentena


**Creación de columna "Peso"**:
A partir del análisis con expertos en negocio, se ha entendido que es importante tomar en cuenta el cambio de gustos de los usuarios a través del tiempo y dar menos peso a prestamos que se realizaron en el pasado a prestamos más recientes. Para esto se creará una nueva columna denominada "Peso" la cual determinará el peso quue tiene dicho prestamo.
La disminución del peso será exponencial según la diferencia con el año actual y se calculará con la siguiente formula:

$Peso = \frac{1}{2^{(2021-20xx)}}$


In [175]:
#Obtenemos el año actual
import datetime
now = datetime.datetime.now()
anio_actual = int(now.year)
print("Anio Actual: ", anio_actual)

Anio Actual:  2021


In [176]:
#Creamos la columna pesos para el dataframe a partir del año en que se realizó el prestamo
join["Peso"] = join.apply(lambda row: 1/2**(anio_actual-row.Year), axis=1 )
join[["Year","Peso"]]

,Year,Peso
0,2019,0.250
1,2019,0.250
2,2019,0.250
3,2019,0.250
4,2019,0.250
...,...,...
489732,2018,0.125
489733,2018,0.125
489734,2018,0.125
489735,2018,0.125


La siguiente función crea la tabla de preferencias y gustos de los usuarios a partir de los prestamos y la fecha de estos(Columna Peso).

Cada Fila representa un usuario, las columnas son los deweys y los datos es el nivel de pertenencia del usuario frente a dicho dewey.

In [177]:
agrupacion = join.groupby(["IDUsuario","DeweyUnidad"])["Peso"].sum().reset_index(name="Peso")
agrupacion.head(5)

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125


In [178]:
#crearTablaPesos: crea la tabla de pesos por usuario
#parámetros
    #columna = {deweyUnidad|deweyDecena|deweyCentena}
def crearTablaPesos(columna):
    #Tabla = pesos_usuarios
        #columnas = deweys
        #filas = usuarios
        #dato = peso que tiene el usuario en dicho dewey
    #creamos las columnas a partir de los deweys diferentes. 
    agrupacion = join.groupby(["IDUsuario",columna])["Peso"].sum().reset_index(name="Peso")
    display(agrupacion.head(5))
    
    #cración del dataframe
    pesos_usuarios = pd.DataFrame()
    #Dataframe auxiliar para acelerar el proceso de concat
    aux = pd.DataFrame()
    
    ids = agrupacion["IDUsuario"].unique()
    print("Total de IDs de usuarios: ", len(ids))
    #Recorremos cada uno de los usuarios
    for usuario in ids:
        #obtenemos todos los prestamos del usuario
        prestamos = agrupacion.loc[agrupacion["IDUsuario"]==usuario]
        columnas = prestamos[columna].values
        pesos = prestamos["Peso"].values
        fila = pd.DataFrame(data = [pesos], columns = columnas)
        fila["IDUsuario"] = usuario
        aux = pd.concat([aux,fila])
        #cada 100 registros concatenamos al dataframe general
        if(aux.shape[0] == 100):
            pesos_usuarios = pd.concat([pesos_usuarios,aux])
            aux = pd.DataFrame()
    pesos_usuarios = pd.concat([pesos_usuarios,aux])
    #Cambiamos los nil por 0
    pesos_usuarios = pesos_usuarios.fillna(0)
    pesos_usuarios.reset_index(drop=True, inplace=True)
    return pesos_usuarios

Se crean 3 tablas de peso diferentes según la columna que se tuvo en cuenta:

{deweyUnidad|deweyDecena|deweyCentena}

In [12]:
pesos_usuarios_unidad=crearTablaPesos("DeweyUnidad")
display(pesos_usuarios_unidad.head(5))
print("Forma de la tabla: ", pesos_usuarios_unidad.shape)

,IDUsuario,DeweyUnidad,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.375
3,000a0630c32a437052696f6921e3181f155291d0,1,0.500
4,000c040064aedd76057c5309064a3823db970644,657,0.125
...,...,...,...
125648,fff735f54719b720fc4c6c723208ad1f06e9605e,986,0.375
125649,fffb2be68fef4b27a810b11a641d84052eb02347,701,0.125
125650,fffb2be68fef4b27a810b11a641d84052eb02347,776,0.125
125651,fffb2be68fef4b27a810b11a641d84052eb02347,781,0.250


Total de IDs de usuarios:  24407


,658,IDUsuario,378,657,1,227,347,350,808,812,...,964,367,383,494,413,496,799,855,876,97
0,0.5,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.375,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,000a0630c32a437052696f6921e3181f155291d0,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,000c040064aedd76057c5309064a3823db970644,0.000,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.000,0.0,0.5,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 756)


In [13]:
pesos_usuarios_decena = crearTablaPesos("DeweyDecena")
display(pesos_usuarios_decena.head(5))
print("Forma de la tabla: ", pesos_usuarios_decena.shape)

,IDUsuario,DeweyDecena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,650,0.125
...,...,...,...
96592,fff735f54719b720fc4c6c723208ad1f06e9605e,980,0.375
96593,fffb2be68fef4b27a810b11a641d84052eb02347,700,0.125
96594,fffb2be68fef4b27a810b11a641d84052eb02347,770,0.125
96595,fffb2be68fef4b27a810b11a641d84052eb02347,780,0.250


Total de IDs de usuarios:  24407


,650,IDUsuario,370,0,220,340,350,800,810,20,...,560,640,470,310,10,480,90,990,50,30
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,0.000,0.0,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 100)


In [14]:
pesos_usuarios_centena = crearTablaPesos("DeweyCentena")
display(pesos_usuarios_centena.head(5))
print("Forma de la tabla: ", pesos_usuarios_centena.shape)

,IDUsuario,DeweyCentena,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.375
3,000a0630c32a437052696f6921e3181f155291d0,0,0.500
4,000c040064aedd76057c5309064a3823db970644,600,0.125
...,...,...,...
61835,fff735f54719b720fc4c6c723208ad1f06e9605e,600,2.625
61836,fff735f54719b720fc4c6c723208ad1f06e9605e,800,2.875
61837,fff735f54719b720fc4c6c723208ad1f06e9605e,900,0.375
61838,fffb2be68fef4b27a810b11a641d84052eb02347,700,0.500


Total de IDs de usuarios:  24407


,600,IDUsuario,300,0,200,800,700,100,400,500,900,-900
0,0.500,00063d52cf68c65d2a569e95c40345c4a305ccc7,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.375,0006b3bdabeaa5389b211c8487bd67d339c97ef7,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,000a0630c32a437052696f6921e3181f155291d0,0.000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.125,000c040064aedd76057c5309064a3823db970644,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,000f23e62e6d4995ede7090c6447ad6736edc4de,1.000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Forma de la tabla:  (24407, 12)


Exportamos e Importamos las tablas para agilizar el proceso

In [13]:
pesos_usuarios_unidad.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_unidad.json')
pesos_usuarios_decena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_decena.json')
pesos_usuarios_centena.to_json(r'C:\Users\user\Downloads\pesos_usuario_x_dewey_centena.json')

In [179]:
pesos_usuarios_unidad = pd.read_json('https://www.dropbox.com/s/aitygqwn9q47rlg/pesos_usuario_x_dewey_unidad.json?dl=1')

In [180]:
pesos_usuarios_decena = pd.read_json('https://www.dropbox.com/s/vr6ehn8xjhojuba/pesos_usuario_x_dewey_decena.json?dl=1')

In [181]:
pesos_usuarios_centena = pd.read_json('https://www.dropbox.com/s/2vnntgjnqpijkgg/pesos_usuario_x_dewey_centena.json?dl=1')

# Normalización de pesos
En busca de que dos usuarios con mismos gustos pero diferente cantidad de prestamos sean equivalentes se decidió  por normalizar cada una de las filas. Así un usuario con 3 prestamos en el dewey 600 y otro con un único prestamo en el dewey 600 se velven equivalente al tener gustos únicamente por este dewey.

In [19]:
#función: normalizar las filas por valores entre cero y uno.
    #Parámetros: pesos_usuario, representa la matriz dispersa que se va a normalizar.
    #{pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
def normalizar_pesos(pesos_usuario):
    print("Normalizando pesos...Iniciando")
    usuarios = pesos_usuario['IDUsuario']
    pesos_usuario = pesos_usuario.apply(pd.to_numeric, errors='coerce').drop(["IDUsuario"],axis=1)
    #display(pesos_usuario)
    #Normalizamos por fila
    sumatoria = pesos_usuario.max(axis=1)
    pesos_norm = pesos_usuario.div(pesos_usuario.sum(axis=1), axis=0)
    pesos_norm_id = pesos_norm.copy()
    # a la tabla le agregamos la columna de IDUsuario para poder identificar que pesos son de cada usuario
    if len(pesos_norm_id) == len(join.IDUsuario.unique()):
        pesos_norm_id['IDUsuario'] = usuarios
    print("Normalizando pesos...Acabado")
    return pesos_norm_id

In [22]:
pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios_unidad)
display(pesos_norm_id_unidad.shape)
pesos_norm_id_unidad.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 756)

,658,378,657,1,227,347,350,808,812,813,...,367,383,494,413,496,799,855,876,97,IDUsuario
0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.0,0.25,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.0,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [23]:
pesos_norm_id_decena = normalizar_pesos(pesos_usuarios_decena)
display(pesos_norm_id_decena.shape)
pesos_norm_id_decena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 100)

,650,370,0,220,340,350,800,810,20,330,...,640,470,310,10,480,90,990,50,30,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [24]:
pesos_norm_id_centena = normalizar_pesos(pesos_usuarios_centena)
display(pesos_norm_id_centena.shape)
pesos_norm_id_centena.head(3)

Normalizando pesos...Iniciando
Normalizando pesos...Acabado


(24407, 12)

,600,300,0,200,800,700,100,400,500,900,-900,IDUsuario
0,1.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00063d52cf68c65d2a569e95c40345c4a305ccc7
1,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0006b3bdabeaa5389b211c8487bd67d339c97ef7
2,0.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000a0630c32a437052696f6921e3181f155291d0


In [389]:
pesos_norm_id_unidad.to_json(r'C:\Users\user\Downloads\pesos_norm_id_unidad.json')


# Agrupación disfusa
Para la agrupación cada usuario podrá pertenecer a uno o mas deweys. Cada usuario tendra un grado de pertenencia a un dewey en especifico dependiendo de sus gustos y preferencias resumidos en las matrices dispersas. 
La función se debe recorrer mínimo 3 veces con diferentes matrices de dispersión:
* pesos_usuarios_unidad
* pesos_usuarios_decena
* pesos_usuarios_centena

In [25]:
#Elimina los valores diferentes de cero para una fila especifica
#Sirve para visualizar únicamente los deweys donde el usuario tiene prestamos. 
def eliminar_cero(id_usuario, df_pesos):
    m1 = (df_pesos['IDUsuario'] == id_usuario)
    m2 = (df_pesos[m1] != 0).all()
    return df_pesos.loc[m1,m2]

In [26]:
eliminar_cero("33c7a2220802831409c62333e13f068363a5d768", pesos_norm_id_unidad)

,1,517,5,IDUsuario
4945,0.333333,0.333333,0.333333,33c7a2220802831409c62333e13f068363a5d768


In [27]:
#Esta función asocia una grado de pertenencia
#Parámetros:
    #pesos_usuarios: matrix dispersa de pesos {pesos_usuarios_unidad|pesos_usuarios_decena|pesos_usuarios_centena}
    #nombre_archivo: nombre del archivo a exportar con la tabla de df_pertenencia
#se crea la tabla df_pertenencia: describe la pertenecia de cada usuario a un cluster(dewey)
def clustering(pesos_usuarios):
    print("Comenzando clustering...")
    #normalizamos los pesos
    pesos_norm_id_unidad = normalizar_pesos(pesos_usuarios)
    #ponemos el usuario a ambos dataframes
    pesos_usuarios["IDUsuario"] = pesos_norm_id_unidad["IDUsuario"]
    #Creamos la estructura del dataframe
    df_pertenencia = pd.DataFrame(columns = ['IDUsuario', 'Cluster', 'Pertenencia','Peso'])
    #iteramos por cada fila(usuario) del dataframe
    for index, row in pesos_norm_id_unidad.iterrows():
        #Extraemos los usuarios
        usuario = row["IDUsuario"]
        #Extraemos los pesos deiferentes de cero normalizados y sin normalizar
        usuario_limpio_norm = eliminar_cero(row["IDUsuario"], pesos_norm_id_unidad)
        usuario_limpio_orig = eliminar_cero(row["IDUsuario"], pesos_usuarios)
        pertenecias = usuario_limpio_norm.drop(["IDUsuario"], axis=1).values
        pesos = usuario_limpio_orig.drop(["IDUsuario"], axis=1).values
        #los clusters son las columnas de estos dataframes
        clusters = usuario_limpio_norm.drop(["IDUsuario"], axis=1).columns
        #creamos un array que tenga las veces necesarias al usuario
        tamanio = len(clusters)
        usuarios = np.repeat([usuario], tamanio)
        #agregamos la fila al dataframe
        fila = pd.DataFrame(data = {'IDUsuario': usuarios, 'Cluster': clusters
                                   , 'Pertenencia': pertenecias[0], 'Peso': pesos[0]})
        df_pertenencia = df_pertenencia.append(fila)
    print("Finalizando clustering...")
    return df_pertenencia

In [28]:
pesos_clustering_unidad = clustering(pesos_usuarios_unidad)
display(pesos_clustering_unidad.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


In [29]:
pesos_clustering_decena = clustering(pesos_usuarios_decena)
display(pesos_clustering_decena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,650,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,650,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,370,0.25,0.125


In [30]:
pesos_clustering_centena = clustering(pesos_usuarios_centena)
display(pesos_clustering_centena.head(3))

Comenzando clustering...
Normalizando pesos...Iniciando
Normalizando pesos...Acabado
Finalizando clustering...


,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,600,1.00,0.500
0,0006b3bdabeaa5389b211c8487bd67d339c97ef7,600,0.75,0.375
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,300,0.25,0.125


In [31]:
pesos_clustering_unidad.reset_index(drop=True, inplace=True)
pesos_clustering_decena.reset_index(drop=True, inplace=True)
pesos_clustering_centena.reset_index(drop=True, inplace=True)

Exportamos e importamos los datos para agilizar el proceso

In [1]:
pesos_clustering_unidad.to_json(r'C:\Users\user\Downloads\pesos_clustering_unidad.json')
pesos_clustering_decena.to_json(r'C:\Users\user\Downloads\pesos_clustering_decena.json')
pesos_clustering_centena.to_json(r'C:\Users\user\Downloads\pesos_clustering_centena.json')

NameError: name 'pesos_clustering_unidad' is not defined

In [5]:
pesos_clustering_unidad = pd.read_json('https://www.dropbox.com/s/6j30n8y3fn8358l/pesos_clustering_unidad.json?dl=1')
pesos_clustering_decena = pd.read_json('https://www.dropbox.com/s/6m7vbpfq8b8qz4s/pesos_clustering_decena.json?dl=1')
pesos_clustering_centena = pd.read_json('https://www.dropbox.com/s/3rjqco5swu55cna/pesos_clustering_centena.json?dl=1')

In [33]:
display(pesos_clustering_unidad.head(3))
print(pesos_clustering_unidad.shape)

,IDUsuario,Cluster,Pertenencia,Peso
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,658,1.00,0.500
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,378,0.25,0.125
2,0006b3bdabeaa5389b211c8487bd67d339c97ef7,657,0.75,0.375


(125653, 4)


# Recomendaciones


## Rankear el material

La función rank_material rankea el material de un dewey en especifico a partir de la experticie de los usuarios dentro de dicho dewey. A partir de los pesos sabemos que tanto sabe un usuario acerca de un dewey especifico. Normalizamos estos valores y a cada libro alquilado que tenga el usuario x al libro x se le da ese puntaje. El puntaje total de un libro es igual a la sumatoria de los puntajes dados por los usuarios. 

In [10]:
#Parámetros:
 #cluster_pertenencia:dataframe con la pertenencia de los usuarios a un cluster.
 #cluster: cluster a evaluar
 #columna: define la columna sobre la cual se va a realizar el join {DeweyUnidad|DeweyDecena|DeweyCentena}
def rank_material(pesos_clustering, cluster, columna):
    #Buscamos únicamente a los usuarios que tienen un grado de pertenencia al cluster.
    usuarios = pesos_clustering.loc[pesos_clustering.Cluster == cluster]
    usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
    #si el peso de algún usuario es mayor a 10 este se deja con el valor de 10
    usuarios.loc[usuarios.Peso > 10, "Peso"] = 10
    
    #normalizar columna de peso
    max_value = usuarios["Peso"].max()
    min_value = usuarios["Peso"].min()
    usuarios["Peso"] = (usuarios["Peso"]) / (max_value)
    #filtramos de la tabla de join los rows con el dewey
    prestamos = join.loc[join[columna] == int(cluster)]
    #display(prestamos)
    #join
    prestamos_data = pd.DataFrame(data=prestamos)
    usuario_data = pd.DataFrame(data=usuarios)
    join_tablas = pd.merge(prestamos_data, usuario_data, left_on='IDUsuario', right_on='IDUsuario', how='inner')
    #finalmente agrupamos al material por llave y hacemos la sumatoria de sus pesos
    materialRank = join_tablas.groupby(["Llave"])["Peso"].sum().reset_index(name="Peso")#Peso o Peso_y
    materialRank = materialRank.sort_values(by=["Peso"],ascending=False)
    return materialRank


In [11]:
material_rankeado = rank_material(pesos_clustering_unidad, 574, "DeweyUnidad")
material_rankeado

<ipython-input-10-5532b9d7b260>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usuarios["Peso"] = usuarios["Peso"].apply(lambda x: float(x))
C:\Users\johan\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-10-5532b9d7b260>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,Llave,Peso
329,722413,68.8375
429,825335,41.4375
522,884195,40.9625
386,788344,40.5250
396,804821,31.2375
...,...,...
373,769277,0.0125
41,269584,0.0125
213,525939,0.0125
532,894506,0.0125


# Circulación de libros
Para cada cluster se recomendarán x libros a cada usuario dependiendo de la pertenencia del usuario al cluster y del nivel del cluster. Por ejemplo si un usuario tiene una pertenencia de 0.6 al cluster 721, y estamos recomendando libros del primer nivel(deweyUnidad) de los cinco libros totales que se recomiendan en este nivel 3 serán del cluster 721. 

Estos x(en este caso 3) libros se seleccionan dentro de los libros mejor valorados del cluster. Para evitar que todos los usuarios tengan las mismas recomendaciones y traten de sacar los mismos libros, antes de recomendar los libros se saca una lista del top libros de cada cluster. De esta lista se seleccionan de manera aleatorio los x libros que necesita el usuario. 

El tamaño de la lista de los libros top de cada dewey se calculará a partir de la cantidad de circulación que tienen cada uno de los deweys en la actualidad(últimos dos años).

In [120]:
def calculo_lista_top_libros(join, columna,baja_circulacion, media_circulacion, alta_circulacion):
    prestamos_recientes = join#.loc[(join.Year == 2020) | (join.Year == 2021)]
    #display(prestamos_recientes)
    frecuencia_prestamos = prestamos_recientes.groupby([columna])["Year"].count().reset_index(name="Frecuencia")
    #display(frecuencia_prestamos.Frecuencia.describe())
    primerPercentil = frecuencia_prestamos.Frecuencia.quantile(0.25)
    print("Percentil 1: ", primerPercentil)
    segundoPercentil = frecuencia_prestamos.Frecuencia.quantile(0.50)
    print("Percentil 2: ", segundoPercentil)
    tercerPercentil = frecuencia_prestamos.Frecuencia.quantile(0.75)
    print("Percentil 3: ", tercerPercentil)
    frecuencia_prestamos["Circulacion"] = 1
    frecuencia_prestamos.loc[(frecuencia_prestamos.Frecuencia <= segundoPercentil), "Circulacion"] = baja_circulacion
    frecuencia_prestamos.loc[(frecuencia_prestamos.Frecuencia > segundoPercentil) &
                             (frecuencia_prestamos.Frecuencia <= tercerPercentil), "Circulacion"] = media_circulacion
    frecuencia_prestamos.loc[frecuencia_prestamos.Frecuencia > tercerPercentil, "Circulacion"] = alta_circulacion
    display(frecuencia_prestamos)
    return frecuencia_prestamos

In [129]:
frecuencia_unidad = calculo_lista_top_libros(join,"DeweyUnidad", 10, 50,100)
frecuencia_decena = calculo_lista_top_libros(join,"DeweyDecena", 50, 100, 150)
frecuencia_centena = calculo_lista_top_libros(join,"DeweyCentena", 100, 150, 200)

Percentil 1:  18.0
Percentil 2:  106.0
Percentil 3:  474.0


,DeweyUnidad,Frecuencia,Circulacion
0,-999,5442,100
1,1,5258,100
2,2,61,10
3,3,297,50
4,4,707,100
...,...,...,...
750,985,303,50
751,986,6172,100
752,987,37,10
753,989,6,10


Percentil 1:  715.5
Percentil 2:  2170.0
Percentil 3:  6692.5


,DeweyDecena,Frecuencia,Circulacion
0,-990,5442,100
1,0,7818,150
2,10,93,50
3,20,3471,100
4,30,11,50
...,...,...,...
94,950,1156,50
95,960,206,50
96,970,1311,50
97,980,8407,150


Percentil 1:  17247.0
Percentil 2:  38558.0
Percentil 3:  63052.0


,DeweyCentena,Frecuencia,Circulacion
0,-900,5442,100
1,0,14559,100
2,100,39986,150
3,200,13117,100
4,300,132220,200
5,400,19935,100
6,500,38558,100
7,600,73846,200
8,700,52258,150
9,800,73959,200


La función **generar recomendación** genera las recomendaciones por nivel para los usuarios de un único cluster teniendo en cuenta el nivel de pertenencia de los usuarios a cada cluster. También general las recomendaciones para libros de baja circulación excluyendo los libros mejores rankeados de cada cluster.
Parámetros:
* pesos_clustering: es la matriz generada por la función de clustering
* cluster: cluster del cual se van a sacar las recomendaciones
* peso_x_nivel: peso del nivel de clustering. Los niveles más especificos tienen mayor peso.
* total_recomendaciones: total de recomendaciones que van a recibir los usuarios
* material_rankeado: dataframe generada por la función rank_material.
* df_frecuencia: dataframe que inidica que tan grande es la circulacion de un dewey para saber seleccionar el tamaño de libros top. 

In [13]:
import math

def generar_recomendacion(pesos_clustering, cluster, peso_x_nivel, total_recomendaciones, material_rankeado, columna):
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel', 'Pertenencia'])
    #buscamos los prestamos asociados unicamente a un cluster
    prestamos_cluster = pesos_clustering.loc[pesos_clustering.Cluster == cluster]
    #Cambiamos el tipo de dato de pertenencia a float
    prestamos_cluster["Pertenencia"] = prestamos_cluster["Pertenencia"].astype(float) 
    #cada row es un usuario diferente
    for index, row in prestamos_cluster.iterrows():
        #calculamos el número de prestamos
        num_prestamos = math.ceil(row["Pertenencia"] * peso_x_nivel * total_recomendaciones)
        #obtenemos los prestamos del usuario de la tabla de join
        #buscamos que estos prestamos no se repitan
        prestamos_usuario = join.loc[(join["IDUsuario"] == row.IDUsuario) 
                                     & (join[columna] == row.Cluster) ]["Llave"].unique()
        tamanio_lista = df_frecuencia.loc[df_frecuencia[columna] == cluster]["Circulacion"].values[0]
        recomendaciones_usuario = material_rankeado[~material_rankeado.Llave.isin(prestamos_usuario)].head(tamanio_lista)
        try:
            llavesRecomendaciones = recomendaciones_usuario["Llave"].sample(n = num_prestamos)
        except:
            llavesRecomendaciones = recomendaciones_usuario["Llave"].head(num_prestamos)
        tamanio = len(llavesRecomendaciones)
        usuarios = np.repeat([row.IDUsuario], tamanio)
        nivel = np.repeat([peso_x_nivel], tamanio)
        pertenencia = np.repeat([row.Pertenencia], tamanio)
        aux_df = pd.DataFrame({'IDUsuario': usuarios,
                               'Llave': llavesRecomendaciones.values,
                               'Nivel': nivel,
                               'Pertenencia': pertenencia})
        recomendaciones = pd.concat([recomendaciones, aux_df], ignore_index = True)
    display("Recomendaciones para el cluster: ", cluster, recomendaciones)
    return recomendaciones

In [111]:
recomendacion_cluster

,IDUsuario,Llave,Nivel,Pertenencia
0,0023eb8457e3722d75e8926cb42f9b7fea38283d,790268,0.5,0.187708
1,0023eb8457e3722d75e8926cb42f9b7fea38283d,261782,BC,0.187708
2,0024fc55a484a66241eba9c79c8532c14a981fd8,488727,0.5,0.166667
3,007dc9488f267e0f74d0fd4f0720280da1ac8e5d,352267,0.5,0.137931
4,009d82769da84f736e209c54556240c2c5fa5776,799677,0.5,0.571429
...,...,...,...,...
3448,ff8164315c407f57567651c4d7d4b83f28a4c69b,818846,0.5,0.544910
3449,ff8164315c407f57567651c4d7d4b83f28a4c69b,825076,BC,0.544910
3450,fff169350c34c6e40763143d1643df3d94698cd4,646265,0.5,0.500000
3451,fff169350c34c6e40763143d1643df3d94698cd4,832974,0.5,0.500000


La función recomendaciones_nivel genera recomendaciones para un nivel completo de clusters. 
Parámetros:
* pesos_clustering: {pesos_clustering_unidad|pesos_clustering_decena|pesos_clustering_centena}
* peso_x_nivel: peso del nivel. Valor entre cero y uno que representa el porcentaje de recomendaciones para dichoi nivel.
* total_recomendaciones: Total de recomendaciones que se le van a hacer a los usuarios.
* df_frecuencia: dataframe que inidica que tan grande es la circulacion de un dewey para saber seleccionar el tamaño de libros top. 

In [125]:
def recomendaciones_nivel(pesos_clustering, peso_x_nivel, total_recomendaciones, columna, df_frecuencia):#"DeweyUnidad"
    print("Comenzando Recomendaciones nivel ")
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
    i=0
    for cluster in pesos_clustering.Cluster.unique():
        #print("CLUSTER: ", cluster)
        material_rankeado = rank_material(pesos_clustering, cluster, "DeweyUnidad")
        #print(material_rankeado)
        lista_recomendaciones = generar_recomendacion(pesos_clustering,
                                                      cluster, peso_x_nivel,
                                                      total_recomendaciones,
                                                      material_rankeado,
                                                      columna,
                                                    df_frecuencia)
        #print(lista_recomendaciones.shape[0])
        recomendaciones = pd.concat([recomendaciones, lista_recomendaciones], ignore_index = True)
        #display(recomendaciones)
        i=i+1
        if i%10 == 0:
            print(i)
    print("Finalizando Recomendaciones nivel")
    return recomendaciones

In [126]:
recomendaciones_final_unidad = recomendaciones_nivel(pesos_clustering_unidad, 0.5, 10, "DeweyUnidad",frecuencia_unidad)
display(recomendaciones_final_unidad)

Comenzando Recomendaciones nivel 
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja cir

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja cir

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja cir

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja cir

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
270
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
340
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
380
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja

No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
490
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
500
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de baja circulacion a recomendar para este usuario
No hay libros de 

,IDUsuario,Llave,Nivel,Pertenencia
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,747540,0.5,1.000000
1,00063d52cf68c65d2a569e95c40345c4a305ccc7,916213,0.5,1.000000
2,00063d52cf68c65d2a569e95c40345c4a305ccc7,819787,0.5,1.000000
3,00063d52cf68c65d2a569e95c40345c4a305ccc7,790166,0.5,1.000000
4,00063d52cf68c65d2a569e95c40345c4a305ccc7,861948,0.5,1.000000
...,...,...,...,...
221747,ba0dd778b3002dc13e76a88bee69727ba978f6b7,721081,0.5,0.006944
221748,dfdc2a0e3c3eecfcb7abe34f5401679590d8439e,721081,0.5,0.114286
221749,da870c1cde89bbc3bdc5972776c9ea818986ede6,730751,0.5,0.041096
221750,c787a2c639990d54d49f9a8f3d19af0440f678f2,312515,0.5,0.054054


In [127]:
recomendaciones_final_decena = recomendaciones_nivel(pesos_clustering_decena, 0.2, 10, "DeweyDecena",frecuencia_decena)


Comenzando Recomendaciones nivel 
10
20
30
40
50
60
70
80
90
Finalizando Recomendaciones nivel


In [130]:
recomendaciones_final_centena = recomendaciones_nivel(pesos_clustering_centena, 0.1, 10, "DeweyCentena",frecuencia_centena)
display(recomendaciones_final_centena)

Comenzando Recomendaciones nivel 
10
Finalizando Recomendaciones nivel


,IDUsuario,Llave,Nivel,Pertenencia
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,313059,0.1,1.000000
1,0006b3bdabeaa5389b211c8487bd67d339c97ef7,313059,0.1,0.750000
2,000c040064aedd76057c5309064a3823db970644,313059,0.1,1.000000
3,00159e086af7d72d80602291964442a68472ed39,313059,0.1,0.615385
4,00186f0d22b83441cca048ddf1e6eab99cc95b7b,313059,0.1,0.100000
...,...,...,...,...
56771,ffa956d8158d97d351f8ee09c9f18aef5289aafb,312877,0.1,0.007092
56772,ffbb81debf44523b4d21e7f48fcda3e4fbb95f4e,312877,0.1,0.071698
56773,ffc9639e31d4edee7f1f489137419d65ea15da5e,312670,0.1,0.600000
56774,ffd24ec2ae25aea321acebccf1e1a08d314461cf,506507,0.1,0.333333


# Recomendaciones libros nuevos

In [133]:
def recomendar_nuevo(usuario, pesos_usuarios, libros_nuevos):
    #obtenemos el dewey sobre el cual el usuario tiene mayor pertenencia
    #sobre este dewey se recomendará el libro nuevo
    mayor_dewey = eliminar_cero(usuario, pesos_usuarios).drop('IDUsuario', axis=1).idxmax(axis=1).values[0]
    mayor_dewey = int(mayor_dewey)
    prestamos_usuario = join.loc[(join["IDUsuario"] == usuario) & (join["DeweyUnidad"] == mayor_dewey) ]["Llave"].unique()
    libros_nuevos_dewey = libros_nuevos.loc[libros_nuevos.DeweyUnidad == mayor_dewey]
    posibles_recomendaciones = libros_nuevos_dewey[~libros_nuevos_dewey.Llave.isin(prestamos_usuario)].head(50)
    try:
        llaveRecomendacion = posibles_recomendaciones["Llave"].sample(n =1).values[0]
    except:
        return -1
    return llaveRecomendacion

In [134]:
def recomendaciones_libros_nuevos(pesos_usuarios):
    usuarios = join.IDUsuario.unique()
    recomendaciones = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel', 'Pertenencia'])
    libros_nuevos = join.loc[(join.FechaCreacion > 2018)]
    for usuario in usuarios:
        llave = recomendar_nuevo(usuario, pesos_usuarios, libros_nuevos)
        if llave != -1:
            aux_df = pd.DataFrame({'IDUsuario': usuario, 'Llave': llave, 'Nivel': "Nuevo", 'Pertenencia': "Nuevo"}, index=[0])
            recomendaciones = pd.concat([recomendaciones, aux_df], ignore_index = True)
    return recomendaciones

In [136]:
recomendaciones_nuevas = recomendaciones_libros_nuevos(pesos_usuarios_unidad)
recomendaciones_nuevas

,IDUsuario,Llave,Nivel,Pertenencia
0,c361c772cd0220bb16dcfb2ad803e4903334ab94,937906,Nuevo,Nuevo
1,87b0e5a61ed712ddfaf5d478ad68c87c825997e9,928131,Nuevo,Nuevo
2,afe8c17ba351d274fc5671978918f0c93032d88f,926106,Nuevo,Nuevo
3,6b39c8ff51a36458e3f68233b82c1ce8bd8cc3d3,310633,Nuevo,Nuevo
4,145f664af916e3bb70e66a166b3e13ce7b70b27b,356130,Nuevo,Nuevo
...,...,...,...,...
22742,a08277a9db1e5cbcac98ca5707a343fce0a8128a,929017,Nuevo,Nuevo
22743,2af3b662cce52945aeec54a03152892f15190444,928334,Nuevo,Nuevo
22744,ea28f5400ec4129b4e3f4a96bc423491043773b5,928315,Nuevo,Nuevo
22745,ba922689e879575fd992d1d92e7dd194d3efdcc7,931456,Nuevo,Nuevo


# Recomendaciones generales

In [138]:
def recomendaciones_usuario(id_usuario, df_recomendaciones):
    recomendaciones_u = df_recomendaciones.loc[df_recomendaciones["IDUsuario"] == id_usuario]
    display(recomendaciones_u)
    return recomendaciones_u

In [139]:
recomendaciones_generales = pd.DataFrame(columns = ['IDUsuario', 'Llave', 'Nivel'])
recomendaciones_generales = pd.concat([recomendaciones_generales, recomendaciones_final_unidad], ignore_index = True)
recomendaciones_generales = pd.concat([recomendaciones_generales, recomendaciones_final_decena], ignore_index = True)
recomendaciones_generales = pd.concat([recomendaciones_generales, recomendaciones_final_centena], ignore_index = True)
recomendaciones_generales = pd.concat([recomendaciones_generales, recomendaciones_nuevas], ignore_index = True)
recomendaciones = recomendaciones_usuario('f238ced57557c11baff818a0a1ede2d60f70b3a7', recomendaciones_generales)

,IDUsuario,Llave,Nivel,Pertenencia
11462,f238ced57557c11baff818a0a1ede2d60f70b3a7,899568,0.5,0.005634
17473,f238ced57557c11baff818a0a1ede2d60f70b3a7,847814,0.5,0.033803
26153,f238ced57557c11baff818a0a1ede2d60f70b3a7,873350,0.5,0.290141
26154,f238ced57557c11baff818a0a1ede2d60f70b3a7,551719,0.5,0.290141
26155,f238ced57557c11baff818a0a1ede2d60f70b3a7,362810,BC,0.290141
30597,f238ced57557c11baff818a0a1ede2d60f70b3a7,705251,0.5,0.011268
54532,f238ced57557c11baff818a0a1ede2d60f70b3a7,631128,0.5,0.098592
61381,f238ced57557c11baff818a0a1ede2d60f70b3a7,765876,0.5,0.011268
65507,f238ced57557c11baff818a0a1ede2d60f70b3a7,692530,0.5,0.002817
67281,f238ced57557c11baff818a0a1ede2d60f70b3a7,346617,0.5,0.005634


In [141]:
recomendaciones_generales.to_json(r'C:\Users\user\Downloads\recomedaciones_generales.json')


In [3]:
recomendaciones_generales = pd.read_json('https://www.dropbox.com/s/jczgx38hbh3l5r7/recomedaciones_generales.json?dl=1')

In [36]:
recomendaciones_generales

,IDUsuario,Llave,Nivel,Pertenencia
0,00063d52cf68c65d2a569e95c40345c4a305ccc7,747540,0.5,1.0
1,00063d52cf68c65d2a569e95c40345c4a305ccc7,916213,0.5,1.0
2,00063d52cf68c65d2a569e95c40345c4a305ccc7,819787,0.5,1.0
3,00063d52cf68c65d2a569e95c40345c4a305ccc7,790166,0.5,1.0
4,00063d52cf68c65d2a569e95c40345c4a305ccc7,861948,0.5,1.0
...,...,...,...,...
403343,a08277a9db1e5cbcac98ca5707a343fce0a8128a,929017,Nuevo,Nuevo
403344,2af3b662cce52945aeec54a03152892f15190444,928334,Nuevo,Nuevo
403345,ea28f5400ec4129b4e3f4a96bc423491043773b5,928315,Nuevo,Nuevo
403346,ba922689e879575fd992d1d92e7dd194d3efdcc7,931456,Nuevo,Nuevo


# Recomnedaciones finales
Debido a la forma en la que se están generando las recomendaciones en este punto es probable que algunos usuarios tengan más de 10 recomendaciones. Para solucionar esto se escogeran por cada nivel el número exacto de recomendaciones que se debían tener por nivel. Esta selección se hará de manera aleatoria dentro de las recomendaciones generales, teniendo en cuenta la alta probabilidad de que al escoger de manera aleatoria sobre un conjunto de datos se mantenga la distribución de los gustos y preferencias del usuario. 

In [41]:
def filtrar_recomendaciones(nivel,total_recomendaciones):
    num_rec = int(nivel*total_recomendaciones)
    recomendaciones_filtradas = pd.DataFrame()
    recomendaciones_nivel = recomendaciones_generales.loc[(recomendaciones_generales.Nivel == nivel)]    
    for usuario in recomendaciones_generales.IDUsuario.unique():
        rec = recomendaciones_nivel.loc[(recomendaciones_nivel.IDUsuario == usuario)]    
        if rec.shape[0] > num_rec:
            rec = rec.sample(n=num_rec)
        recomendaciones_filtradas = pd.concat([recomendaciones_filtradas,rec], ignore_index = True)
    return recomendaciones_filtradas

In [42]:
recomendaciones_finales_unidad = filtrar_recomendaciones(0.5,10)

In [44]:
recomendaciones_finales_decena = filtrar_recomendaciones(0.2,10)

In [45]:
recomendaciones_finales_centena = filtrar_recomendaciones(0.1,10)

In [13]:
recomendaciones_generales.Nivel.unique()

array([0.5, 'BC', 0.2, 0.1, 'Nuevo'], dtype=object)

In [22]:
recomendaciones_bc = recomendaciones_generales.loc[recomendaciones_generales.Nivel == "BC"]
recomendaciones_nuevas = recomendaciones_generales.loc[recomendaciones_generales.Nivel == "Nuevo"]




In [46]:
recomendaciones_finales = pd.DataFrame()
recomendaciones_finales = pd.concat([recomendaciones_finales, recomendaciones_finales_unidad], ignore_index = True)
recomendaciones_finales = pd.concat([recomendaciones_finales, recomendaciones_finales_decena], ignore_index = True)
recomendaciones_finales = pd.concat([recomendaciones_finales, recomendaciones_finales_centena], ignore_index = True)
recomendaciones_finales = pd.concat([recomendaciones_finales, recomendaciones_nuevas], ignore_index = True)
recomendaciones_finales = pd.concat([recomendaciones_finales, recomendaciones_bc], ignore_index = True)





In [52]:
recomendaciones_finales.to_json(r'/Users/juansebastianangaritatorres/Downloads/recomedaciones_finales.json')



In [47]:
recomendaciones_data = pd.DataFrame(data=recomendaciones_finales)
join_data = pd.DataFrame(data=join)
recomendaciones_completas = pd.merge(recomendaciones_data, join_data, left_on='Llave', right_on='Llave', how='inner')

In [48]:
recomendaciones_completas.loc[recomendaciones_completas.IDUsuario_x == "6466dbb15c41fdacb59eb1179817958de2c57191"].Titulo.unique()

array(['El lector común Virginia Woolf ; selección, traducción y notas de Daniel Nisa Cáceres con la colaboración de Ana Pérez Vega',
       'Ulises James Joycre',
       'Cuentos macabros Edgar Allan Poe ; traducidos por Julio Cortázar ; ilustrados por Benjamin Lacombe',
       'Olivia salva el circo escrito e ilustrado por Ian Falconer ; traducción de Natalia Cervantes',
       'El codigo Da Vinci Dan Brown ; traducción Juanjo Estrella',
       'Dispositivos electrónicos y amplificación de señales Adel S. Sedra y Kenneth C. Smith ; traducción Denio A. Topete E.',
       'Introducción al estudio de la literatura Franco Brioschi y Constanzo di Girolamo',
       'Historia de la literatura alemana por Rodolfo E. Modern',
       'Moby Dick Herman Melville ; adaptación de Jean Rouaud ; ilustraciones de Denis Deprez ; traducción de Julián Meza',
       'El socio John Grisham'], dtype=object)

In [382]:
recomendaciones_completas.l["Titulo"].unique()

array(['Estrategias eficaces de ventas el sistema más contrastado de ideas, métodos y técnicas de ventas empleado por los líderes comerciales mundiales Brian Tracy ; traducción Guillermo Sánchez',
       'La biblia de JavaScript Lázaro Issi Camy',
       'Relatos ineditos Ernest Hemingway',
       'Ángel mecánico Cassandra Clare ; traducción Patricia Nunes',
       'Westward the sun Geoffrey Cotterell',
       'Los grandes conflictos sociales y económicos de nuestra historia Indalecio Liévano Aguirre ; editor Alberto Ramírez Santos',
       'Franz Kafka o la soledad Marthe Robert',
       'The common place of law stories from everyday life Patricia Ewick & Susan S. Silbey',
       'Economía de la agricultura Jesus Antonio Bejarano Avila',
       'Criminología un enfoque humanístico Jorge Restrepo Fontalvo',
       'Vida la ciencia de la biología William Kirkwood Purves...[et al.] ; traducción Gabriela Hermitte y Juan J. García Fernández',
       'El negociador Frederick Forsyth ; tradu